# script Instagram à partir du 12 octobre 2019

## A) import des librairies
    supprimer celles qui ne seront pas utiliées au final

In [1]:
from pandas.io.json import json_normalize
import json
from instagram.client import InstagramAPI
import requests
from bs4 import BeautifulSoup
import pandas as pd

from lxml import html

import urllib.request
from urllib.request import urlopen

import re

from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
from datetime import datetime

import csv

## B) Préparation du fichier CSV pour scrap 
1) je récupère la liste des id de FB dans le fichier BDD pour scrap en l'enregistrant sous CSV <br />
2) je renomme le fichier 'INS_USERS_POUR SCRIPT' et l'ouvre<br />
3) je m'assure de supprimer les redondances <br />
4) je lance ensuite de script de scraping <br />
5) je relance la requete pour les données vides avec le fichier 'INS_USERS_POUR SCRIPT-suite'

In [47]:
insta = pd.read_csv('LISTE_INS_USERS_POUR SCRIPT_SHORT_688-850.csv',encoding='utf-8',sep=',')
username=insta.url_rs_ID.unique()

## C) Scrap Avec Selenium et traitement

In [48]:
%%time

insta_publi_final = pd.DataFrame() 

browser = webdriver.Firefox()

def getPubli(user):
    
    #delays = [7, 4, 6, 2, 10, 19]
      
    global insta_publi_temp
    try:
        # 2) Scrap Avec Selenium
        browser.get('https://www.instagram.com/'+user+'/?hl=fr')
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        source = browser.page_source
        data=BeautifulSoup(source, 'lxml')

        # 3) récupération de la partie du code qui contient les infos necessaire : balise script
        text_trend =[element.text for element in data.find_all("script") if element.text.startswith('window._sharedData')==True]
        
        #delay = np.random.choice(delays)
        time.sleep(3)

        # 4) Je passe le code récupéré au format JSON puis le transforme en Dataframe
        for x in text_trend:
            page_json = x.split(' = ', 1)[1].rstrip(';')
            data = json.loads(page_json)
            df=pd.DataFrame(data)

        return df
    except :
        #print(x)
        pass
entry_data =pd.DataFrame()
#username=['reachread','venice_bliss']

"""   try:  
        u = driver.find_elements_by_id("data")  
        text = u[0].get_attribute("innerHTML")  
        driver.find_elements_by_class_name("aclassname")[0].click()  
   except exceptions.StaleElementReferenceException,e:
        print(e)
        pass  """


for user in username:
    test = getPubli(user)
    # 5) Dans ce Dataframe seul le dict qui se trouve dans la colonne 'entry_data' et l'index 'ProfilePage' m'intéresse¶
        #Je récupère donc la cellulle qui contient les infos du compte (infos générales et publications) et nomme la variable 'entry_data'
    entry_data_temp=test[['entry_data']]
    entry_data_temp = json_normalize(entry_data_temp['entry_data']['ProfilePage'])
    entry_data =entry_data.append(entry_data_temp,ignore_index=True)
entry_data.to_json('RECUEIL_PAR_JOUR/entry_data_backup_688-850.json', orient='records')

browser.close()

CPU times: user 11 s, sys: 678 ms, total: 11.7 s
Wall time: 12min 29s


## Traitement du scraping

In [49]:
entry_data = pd.read_json('RECUEIL_PAR_JOUR/entry_data_backup_688-850.json')
entry_data.head(3)

,graphql.user.biography,graphql.user.blocked_by_viewer,graphql.user.business_category_name,graphql.user.connected_fb_page,graphql.user.country_block,graphql.user.edge_felix_video_timeline.count,graphql.user.edge_felix_video_timeline.edges,graphql.user.edge_felix_video_timeline.page_info.end_cursor,graphql.user.edge_felix_video_timeline.page_info.has_next_page,graphql.user.edge_follow.count,...,graphql.user.is_private,graphql.user.is_verified,graphql.user.profile_pic_url,graphql.user.profile_pic_url_hd,graphql.user.requested_by_viewer,graphql.user.username,logging_page_id,show_follow_dialog,show_suggested_profiles,toast_content_on_load
0,Compte officiel des Editions Rivages #rivagesn...,False,Publishers,NaN,False,0,[],None,False,499,...,False,False,https://scontent-cdg2-1.cdninstagram.com/vp/96...,https://scontent-cdg2-1.cdninstagram.com/vp/ed...,False,editionsrivages,profilePage_4180835495,False,False,NaN
1,Compte tenu par @astridlafleur\n#fantastique #...,False,Publishers,NaN,False,0,[],None,False,134,...,False,False,https://scontent-cdg2-1.cdninstagram.com/vp/39...,https://scontent-cdg2-1.cdninstagram.com/vp/65...,False,rebelleeditions,profilePage_1426111705,False,True,NaN
2,Éditeur indépendant de littérature contemporai...,False,Home Services,NaN,False,0,[],None,False,369,...,False,False,https://scontent-cdg2-1.cdninstagram.com/vp/a4...,https://scontent-cdg2-1.cdninstagram.com/vp/94...,False,publienet,profilePage_4934540199,False,False,NaN


In [50]:
%%time
# 6) infos de base du compte : nb de followers et nb de publications
    #Ici je récupère les infos relative au compte : nom de la page, id, nb de followers et nb de publications
insta_info=entry_data[['graphql.user.username','logging_page_id','graphql.user.edge_followed_by.count','graphql.user.edge_owner_to_timeline_media.count']]

# 7) Travail sur les publications
    #Je récupre le texte des publications depuis la colonne 'node.edge_media_to_caption.edges'
insta_publi_temp=entry_data['graphql.user.edge_owner_to_timeline_media.edges'].apply(lambda x:json_normalize(x))

insta_publi=pd.DataFrame()
for x in insta_publi_temp:
    insta_publi =insta_publi.append(x,ignore_index=True, sort=False)

insta_publi = insta_publi.assign(text_publication_temp=insta_publi['node.edge_media_to_caption.edges'].apply(lambda x: json_normalize(x)))

df = pd.DataFrame(['Pas de texte'],columns=['node.text'])
insta_publi['text_publication_temp']=insta_publi['text_publication_temp'].apply(lambda x: pd.DataFrame(df) if x.empty == True else x)

#texte publication en dataframe=> OK

insta_publi.loc[:,'text_publication'] = [insta_publi['text_publication_temp'][i]['node.text'][0] for i in range(len(insta_publi))]
#texte publication clean => OK

# test point 7 => OK

#8) travail sur la colonne Timestamp pour récupérer les infos de date (jour/mois/année) et d'heure
insta_publi = insta_publi.assign(date_time=insta_publi['node.taken_at_timestamp'].map(lambda x: datetime.fromtimestamp(x)))
insta_publi = insta_publi.assign(date=insta_publi['date_time'].map(lambda x: datetime.date(x)))
insta_publi = insta_publi.assign(code_jour_semaine=insta_publi['date_time'].map(lambda x: datetime.date(x).isoweekday()))
insta_publi = insta_publi.assign(time=insta_publi['date_time'].map(lambda x: datetime.time(x)))

insta_publi['day'] = [d.day for d in insta_publi['date']]
insta_publi['month'] = [d.month for d in insta_publi['date']]
insta_publi['year'] = [d.year for d in insta_publi['date']]

# test point 8 => OK

# 9) Travail sur les hashtag
    #a) je récupère les # présents dans les publications et les mets dans une nouvelle colonne nommée 'hashtag'<br/>
    #b) je crée une autre colonne qui affiche le nombre de hashtags
def findHastag(publi):
    return [hashtag for hashtag in publi.split(' ') if('#' in hashtag)]

insta_publi = insta_publi.assign(hashtag=insta_publi['text_publication'].map(lambda publi: findHastag(publi.replace('\n',''))))
insta_publi = insta_publi.assign(hashtag_count=insta_publi['hashtag'].map(lambda publi: len(publi)))

# test point 9 => OK

CPU times: user 3.92 s, sys: 83.8 ms, total: 4 s
Wall time: 4.04 s


In [51]:
insta_publi.head(3)
#insta_publi.columns

,node.__typename,node.accessibility_caption,node.comments_disabled,node.dimensions.height,node.dimensions.width,node.display_url,node.edge_liked_by.count,node.edge_media_preview_like.count,node.edge_media_to_caption.edges,node.edge_media_to_comment.count,...,text_publication,date_time,date,code_jour_semaine,time,day,month,year,hashtag,hashtag_count
0,GraphSidecar,L’image contient peut-être : une personne ou p...,False,1080,1080,https://scontent-cdg2-1.cdninstagram.com/vp/3a...,70,70,"[{'node': {'text': '🖊 Ce week-end, Stuart Nevi...",0,...,"🖊 Ce week-end, Stuart Neville sera au festival...",2019-11-13 11:08:38,2019-11-13,3,11:08:38,13,11,2019,"[#rencontres, #signatures, #debats, #cafélitté...",6
1,GraphImage,Aucune description de photo disponible.,False,1080,1080,https://scontent-cdg2-1.cdninstagram.com/vp/4d...,70,70,"[{'node': {'text': 'Ellroy, agenda modifié ! L...",0,...,"Ellroy, agenda modifié !\nL'événement et la dé...",2019-11-12 17:36:57,2019-11-12,2,17:36:57,12,11,2019,"[75015.#jamesellroy, #latempetequivient, #even...",6
2,GraphImage,Aucune description de photo disponible.,False,1080,1080,https://scontent-cdg2-1.cdninstagram.com/vp/8c...,94,94,[{'node': {'text': '« Jours d’hiver » de Berna...,0,...,« Jours d’hiver » de Bernard Maclaverty et « L...,2019-10-31 17:33:59,2019-10-31,4,17:33:59,31,10,2019,"[#Rivages, #Petitebibliothequerivages, #lemond...",4


In [52]:
#10) Infos relatives au dataframe

insta_publi.shape, insta_publi.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 44 columns):
node.__typename                             1718 non-null object
node.accessibility_caption                  1613 non-null object
node.comments_disabled                      1718 non-null bool
node.dimensions.height                      1718 non-null int64
node.dimensions.width                       1718 non-null int64
node.display_url                            1718 non-null object
node.edge_liked_by.count                    1718 non-null int64
node.edge_media_preview_like.count          1718 non-null int64
node.edge_media_to_caption.edges            1718 non-null object
node.edge_media_to_comment.count            1718 non-null int64
node.fact_check_information                 0 non-null object
node.fact_check_overall_rating              0 non-null object
node.gating_info                            0 non-null object
node.id                                     1718 non-null object

((1718, 44), None)

In [53]:
insta_info.tail(3)

,graphql.user.username,logging_page_id,graphql.user.edge_followed_by.count,graphql.user.edge_owner_to_timeline_media.count
151,librairiedesmarquises,profilePage_5454333472,206,5
152,maupetit_100ans,profilePage_9417161266,239,19
153,librairiepecari,profilePage_4786547459,249,90


In [54]:
insta_info.to_json('RECUEIL_PAR_JOUR/insta_info_backup_688-850.json', orient='records')

In [55]:
insta_publi.to_json('RECUEIL_PAR_JOUR/insta_publi_backup_688-850.json', orient='records')

### TO DO = 
- penser à faire du multipocessing pour baisser le temps de traitement : http://localhost:8888/notebooks/IronHack/data-labs/module-1/lab-parallelization/your-code/main.ipynb